## Loading Financial Data

In [27]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler                # Scale the data between 0 and 1
from tensorflow.keras.layers import Dense, Dropout, LSTM      # Long Short Term Memory
from tensorflow.keras.models import Sequential

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

ImportError: SystemError: <built-in method __contains__ of dict object at 0x000001E19C14DF80> returned a result with an error set

In [6]:
crypto_currency = 'BTC'
against_currency = 'USD'

In [7]:
start = dt.datetime(2015,1,1)
end = dt.datetime.now()

In [9]:
data = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', start, end)

In [26]:
data.head()

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-01-01,320.434998,314.002991,320.434998,314.248993,8036550,314.248993
2015-01-02,315.838989,313.565002,314.079010,315.032013,7860650,315.032013
2015-01-03,315.149994,281.082001,314.846008,281.082001,33054400,281.082001
2015-01-04,287.230011,257.612000,281.145996,264.195007,55629100,264.195007
2015-01-05,278.341003,265.084015,265.084015,274.473999,43962800,274.473999


In [25]:
data.sort_values(by = ['Date'], ascending = False)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-04-17,40549.304688,40200.234375,40413.757812,40228.894531,15948793856,40228.894531
2022-04-16,40633.679688,40078.425781,40552.316406,40424.484375,16833150693,40424.484375
2022-04-15,40617.722656,39866.832031,39939.402344,40553.464844,21756855753,40553.464844
2022-04-14,41451.480469,39695.746094,41160.218750,39935.515625,24342001973,39935.515625
2022-04-13,41430.054688,39712.746094,40123.570312,41166.730469,27691105228,41166.730469
...,...,...,...,...,...,...
2015-01-05,278.341003,265.084015,265.084015,274.473999,43962800,274.473999
2015-01-04,287.230011,257.612000,281.145996,264.195007,55629100,264.195007
2015-01-03,315.149994,281.082001,314.846008,281.082001,33054400,281.082001


## Data Preparation

In [17]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1,1))

In [18]:
# Number of days where the prediction is based on
prediction_days = 60

In [19]:
# Train Data
x_train, y_train = [], []

for x in range(prediction_days, len(scaled_data)):
    x_train.append(scaled_data[x-prediction_days: x, 0])
    y_train.append(scaled_data[x,0])

In [21]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

## Neural Network Model

In [ ]:
model = Sequential()

In [ ]:
# Layers
model.add(LSTM(units = 50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(Dropout(0.2))   # Prevents Overfiting
model.add(LSTM(units = 50, return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout = 0.2)
model.add(Dense(units=1))

In [ ]:
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, epochs = 25, batch_size = 32)

## Testing the Model

In [ ]:
test_start = dt.datetime(2020)   # Starts date of the data
test_end = dt.datetime.now()

In [ ]:
test_data = web.DataReader(f'{crypto_currency}-{against_currency}', 'yahoo', test_start, test_end)
actual_prices = test_data['Close'].values

In [ ]:
total_dataset = pd.concat((data['Close'], test_data['Close'], axis = 0))

In [ ]:
model_inptus = total_dataset[len(total_dataset) - len(test_data) - prediction_days:].values
model_inptus = model_inputs.reshape(-1, 1)
model_inptus = scaler.fit_transform(model_inputs)

In [ ]:
x_test = []

In [ ]:
for x in range(prediction_days, len(model_inputs)):
    x_test.append(model_inputs[x-prediction_days: x, 0])

In [ ]:
x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [ ]:
# Predict Prices
prediction_prices = model.predic(x_test)
prediction_proces = scaler.inverse_transform(prediction_prices)

In [ ]:
# Plots
plt.plot(actual_prices, color = 'black', label = 'Actual Prices')
plt.plot(prediction_prices, color = 'green', label = 'Predocted Prices')
plt.title(f'{crypto_currency} price prediction')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend(loc = 'upper left')
plt.show()

## Price Forecasting

In [ ]:
real_data = [model_inputs[len(model_inputs) + 1 - prediction_days:len(model_inputs) + 1, 0]]
real_data = np.array(real_data)
real_data = np.reshape(real_data, (real_data.shape[0], real_data.shape[1], 1))

In [ ]:
prediction = model.predict(real_data)
prediction = scaler.inverse_transform(prediction)
print()